<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
세바스찬 라시카(Sebastian Raschka)가 쓴 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a>의 번역서 <br><<b><a href="<a href="http://tensorflow.blog/llm-from-scratch">밑바닥부터 만들면서 배우는 LLM</a></b>>의 예제 코드입니다.<br>
<br>코드 저장소: <a href="https://github.com/rickiepark/llm-from-scratch">https://github.com/rickiepark/llm-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://tensorflow.blog/llm-from-scratch"><img src="https://tensorflowkorea.wordpress.com/wp-content/uploads/2025/09/ebb091ebb094eb8ba5llm_ebb3b8ecb185_ec959eeba9b4.jpg" width="100px"></a>
</td>
</tr>
</table>


<a href="https://colab.research.google.com/github/rickiepark/llm-from-scratch/blob/main/ch02/01_main-chapter-code/ch02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2장: 텍스트 데이터 다루기

이 노트북에서 사용할 패키지:

In [2]:
from importlib.metadata import version

pkgs = [
        "tiktoken",
        "torch",
        "transformers"
       ]
for p in pkgs:
    print(f"{p} 버전: {version(p)}")

tiktoken 버전: 0.12.0
torch 버전: 2.8.0+cu126
transformers 버전: 4.57.3


- 이 장은 LLM의 입력 데이터를 준비하기 위한 데이터 전처리와 샘플링을 다룹니다.

<img src="images/llm_from_scratch/ch02_compressed/01.webp" width="800px">

## 2.1 단어 임베딩 이해하기

- 많은 종류의 임베딩이 있지만 이 책에서는 텍스트 임베딩에 초점을 맞춥니다.

<img src="images/llm_from_scratch/ch02_compressed/02.webp" width="700px">

- LLM은 (수천 개의 차원에 달하는) 고차원 공간의 임베딩을 다룹니다.
- 사람은 3차원 이하만 인식하고 고차원 공간을 시각화할 수 없기 때문에 아래 그림에서는 2차원 임베딩 공간을 보여줍니다.

<img src="images/llm_from_scratch/ch02_compressed/03.webp" width="500px">

## 2.2 텍스트 토큰화하기

- 이 절에서 텍스트를 토큰화합니다. 즉 텍스트를 개별 단어와 구둣점 문자 같은 더 작은 단위로 나눕니다.

<img src="images/llm_from_scratch/ch02_compressed/04.webp" width="500px">

- 사용할 텍스트 데이터는 이디스 워튼(Edith Wharton)의 단편 소설 "[The Verdict](https://en.wikisource.org/wiki/The_Verdict)"입니다.

In [3]:
import os
import requests

if not os.path.exists("datas/the-verdict.txt"):
    url = (
        "https://raw.githubusercontent.com/rasbt/"
        "LLMs-from-scratch/main/ch02/01_main-chapter-code/"
        "the-verdict.txt"
    )
    file_path = "datas/the-verdict.txt"

    response = requests.get(url, timeout=30)
    response.raise_for_status()
    with open(file_path, "wb") as f:
        f.write(response.content)

In [4]:
with open("datas/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print("총 문자 개수:", len(raw_text))
print(raw_text[:99])

총 문자 개수: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


- LLM을 위해 이 텍스트를 토큰화하고 임베딩하는 것이 목표입니다.
- 샘플 텍스트를 사용해 간단한 토크나이저를 만들고 나중에 위 텍스트에 적용해 보겠습니다.
- 다음 정규 표현식은 공백을 기준으로 텍스트를 나눕니다.

In [5]:
import re

text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)

print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


- 공백으로만 나누지 않고 쉼표나 마침표도 나누고 싶습니다. 이를 위해 정규식을 수정해 보죠.

In [6]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


- 결과에 빈 문자열이 포함되어 있으므로 이를 삭제합니다.

In [7]:
# 각 항목에서 공백을 삭제하고 빈 문자열을 제외합니다.
result = [item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


- 출력 결과가 좋아 보입니다. 추가적으로 물음표 같은 다른 구둣점 문자도 처리해 보죠.

In [8]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


- 좋네요. 이제 이 토큰화를 원시 텍스트에 적용해 보겠습니다.

<img src="images/llm_from_scratch/ch02_compressed/05.webp" width="500px">

In [9]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


- 총 토큰 개수를 확인해 보죠.

In [10]:
print(len(preprocessed))

4690


## 2.3 토큰을 토큰 ID로 변환하기

- 그다음 텍스트 토큰을 나주에 임베딩 층에서 처리할 수 있는 토큰 ID로 변환합니다.

<img src="images/llm_from_scratch/ch02_compressed/06.webp" width="700px">

- 고유한 토큰으로 구성된 어휘사전을 만듭니다.

In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

1130


In [12]:
vocab = {token:integer for integer,token in enumerate(all_words)}

- 어휘사전에 있는 처음 50개 항목을 확인합니다:

In [13]:
for i, item in enumerate(vocab.items()):
    print(item)
    if i >= 50:
        break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


- 아래 그림은 작은 어휘사전을 사용해 샘플 텍스트를 토큰화하는 과정을 보여줍니다:

<img src="images/llm_from_scratch/ch02_compressed/07.webp" width="700px">

- 이를 모두 합쳐서 토큰화 클래스를 만듭니다.

In [14]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text) # 'hello,. world'

        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 구둣점 문자 앞의 공백을 삭제합니다.
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

- `encode` 함수는 텍스트를 토큰 ID로 바꿉니다.
- `decode` 함수는 토큰 ID를 텍스트로 바꿉니다.

<img src="images/llm_from_scratch/ch02_compressed/08.webp" width="700px">

- 토크나이저를 사용해 텍스트를 정수로 인코딩(즉, 토큰화)합니다.
- 이 정수를 (나중에) 임베딩하여 LLM의 입력으로 사용할 수 있습니다.

In [15]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


- 정수를 텍스트로 다시 디코딩할 수 있습니다.

In [16]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [17]:
tokenizer.decode(tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4 특수 문맥 토큰 추가하기

- 알지 못하는 단어와 텍스트의 끝을 알리는 "특수" 토큰을 추가하면 도움이 됩니다.

<img src="images/llm_from_scratch/ch02_compressed/09.webp" width="700px">

- 일부 토크나이저는 특수 토큰을 사용해 LLM이 추가적인 맥락을 이해하도록 돕습니다.
- 이런 특수 토큰에는 다음과 같은 것들이 있습니다.
  - `[BOS]` (beginning of sequence)는 텍스트의 시작을 표시합니다
  - `[EOS]` (end of sequence)는 텍스트의 끝을 표시합니다(서로 다른 두 개의 위키백과 문서나 책과 같이 관련이 없는 여러 텍스트를 연결할 때 사용합니다.)
  - `[PAD]` (padding) 하나 이상의 배치 크기로 LLM을 훈련할 때 배치 안에 길이가 다른 텍스트가 포함될 수 있습니다. 모든 텍스트의 길이를 동일하게 맞추기 위해 짧은 텍스트를 패딩 토큰을 사용해 배치에서 가장 긴 텍스트의 길이까지 확장합니다.
- `[UNK]`는 어휘 사전에 없는 단어를 나타냅니다.
- GPT-2는 위와 같은 토큰이 필요 없으며 복잡도를 줄이기 위해 `<|endoftext|>` 토큰만 사용합니다.
- `<|endoftext|>`는 위에서 언급한 `[EOS]` 토큰과 같습니다.
- GPT는 `<|endoftext|>`를 패딩에도 사용합니다(배치 입력에서 모델을 훈련할 때 마스크를 사용하기 때문에 패딩된 토큰에는 주의를 기울이지 않아 어떤 토큰을 사용하던지 관계가 없습니다).
- GPT-2는 어휘사전에 없는 토큰을 위해 `<UNK>` 토큰을 사용하지 않습니다. 대신 GPT-2는 바이트 페어 인코딩(BPE) 토크나이저를 사용해 단어를 부분단어로 분할합니다. 이에 대해서는 나중에 설명하겠습니다.

- 두 개의 독립된 텍스트 사이에 `<|endoftext|>` 토큰을 사용합니다:

<img src="images/llm_from_scratch/ch02_compressed/10.webp" width="800px">

- 다음과 같은 텍스트를 토큰화해 보죠:

In [18]:
tokenizer = SimpleTokenizerV1(vocab)

text = "Hello, do you like tea. Is this-- a test?"

tokenizer.encode(text)

KeyError: 'Hello'

- 단어 "Hello"가 어휘사전에 없기 때문에 오류가 발생합니다.
- 이를 처리하기 위해 알지 못하는 단어를 표현하는 `"<|unk|>"` 같은 특수 토큰을 어휘 사전에 추가할 수 있습니다.
- 어휘사전을 준비했으므로 GPT-2 훈련에서 텍스트의 끝을 나타내기 위해 사용된 `"<|endoftext|>"` 토큰을 추가하겠습니다(또한 훈련 데이터셋이 여러 개의 글이나 책으로 구성된 경우 연결된 텍스트 사이를 나타내기 위해서도 사용됩니다).

In [19]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [20]:
len(vocab.items())

1132

In [21]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


- 새로운 `<unk>` 토큰을 사용할 때와 방법을 알려 주기 위해 토크나이저를 수정합니다.

In [22]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 구둣점 문자 앞의 공백을 삭제합니다.
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

수정된 토크나이저로 텍스트를 토큰화해 보죠:

In [23]:
tokenizer = SimpleTokenizerV2(vocab)

text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."

text = " <|endoftext|> ".join((text1, text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [24]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [25]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

## 2.5 BPE Tokenizer

- BPE 토크나이저는 기본 단위(단일 문자)의 리스트로 시작해서 가장 빈번하게 함께 등장하는 토큰을 합쳐 어휘사전에 새롭게 추가하는 방식으로 원하는 어휘사전 크기에 도달할 때까지 점진적으로 새 토큰 만드는 과정을 반복합니다.  
아래 실습에서는 논문(https://arxiv.org/pdf/1508.07909.pdf) 에 언급된 알고리즘을 그대로 구현해 보겠습니다.

In [26]:
import re, collections
from IPython.display import display, Markdown, Latex

In [27]:
num_merges = 10

In [28]:
vocab = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>' : 6,
         'w i d e s t </w>' : 3
         }

In [29]:
import collections
import re
from IPython.display import display, Markdown

def get_stats(vocab):
    """
    현재 단어장(vocab)에서 인접한 문자 쌍(Bigram)의 빈도수를 계산하여 반환하는 함수
    """
    pairs = collections.defaultdict(int) # 쌍의 빈도수를 저장할 딕셔너리 (기본값 0)
    
    for word, freq in vocab.items():
        symbols = word.split() # 단어 내의 문자(심볼)들을 공백 기준으로 분리 (예: "l o w" -> ['l', 'o', 'w'])
        
        # 현재 단어 안에서 인접한 모든 쌍을 순회
        for i in range(len(symbols)-1):
            # (현재 글자, 다음 글자)를 키로 하여 빈도수를 누적
            pairs[symbols[i], symbols[i+1]] += freq
            
    print('Frequences :', dict(pairs)) # (디버깅용) 계산된 쌍들의 빈도수 출력
    return pairs

def merge_vocab(pair, v_in):
    """
    가장 많이 등장한 문자 쌍(pair)을 찾아, 단어장(v_in) 내에서 하나의 문자로 병합하는 함수
    """
    v_out = {}
    # 병합할 쌍을 정규표현식에서 사용할 수 있도록 이스케이프 처리하고 공백으로 연결 (예: ('e', 's') -> "e\ s")
    bigram = re.escape(' '.join(pair))
    
    # 정규표현식 컴파일:
    # (?<!\S) : 앞에 공백이 아닌 문자가 없어야 함 (단어의 시작이거나 공백 뒤여야 함)
    # (?!\S)  : 뒤에 공백이 아닌 문자가 없어야 함 (단어의 끝이거나 공백 앞이어야 함)
    # 즉, 현재 분리된 토큰 상태에서 정확히 해당 bigram만 매칭하기 위함
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    
    for word in v_in:
        # 매칭된 bigram을 병합된 형태(공백 제거)로 치환
        # 예: "e s" -> "es"
        w_out = p.sub(''.join(pair), word)
        
        # 병합된 새 단어를 키로, 기존 빈도수를 값으로 저장
        v_out[w_out] = v_in[word]
        
    return v_out

# BPE 병합 규칙을 저장할 딕셔너리들
bpe_codes = {}          # 병합된 토큰 -> 순위(iteration 번호)
bpe_codes_reverse = {}  # 병합된 결과 문자열 -> 원래 쌍(pair)

# num_merges 횟수만큼 BPE 알고리즘 반복 수행
for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1))) # 진행 상황 출력 (Jupyter 환경)
    
    # 1. 현재 단어장에서 모든 쌍의 빈도수 계산
    pairs = get_stats(vocab)
    
    # 2. 가장 빈도수가 높은(가장 많이 등장한) 쌍 찾기
    best = max(pairs, key=pairs.get)
    
    # 3. 단어장에서 해당 쌍을 병합하여 업데이트
    vocab = merge_vocab(best, vocab)

    # 4. 병합 규칙 저장
    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    # 결과 출력
    print("New merge: {}".format(best)) # 이번 턴에 병합된 쌍
    print("Vocabulary: {}".format(vocab)) # 업데이트된 단어장 상태

### Iteration 1

Frequences : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
New merge: ('e', 's')
Vocabulary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

Frequences : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
New merge: ('es', 't')
Vocabulary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

Frequences : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge: ('new', 'est</w>')
Vocabulary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

Frequences : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge: ('low', '</w>')
Vocabulary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

Frequences : {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
New merge: ('w', 'i')
Vocabulary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


- 코드의 핵심 요약

1.  **`get_stats`**: 현재 분리된 상태의 단어들(예: `l o w`)을 보고, 붙어 있는 글자 쌍(`l o`, `o w`)이 총 몇 번 나오는지 셉니다.
2.  **`merge_vocab`**: 가장 자주 나온 쌍(Best Pair)을 골라, 단어장에서 그 두 글자를 하나로 합칩니다. (예: `e`와 `s`가 가장 많았다면, 모든 `e s`를 `es`로 바꿈).
3.  **반복문**: 이 과정을 `num_merges`만큼 반복하여, 자주 나오는 문자 패턴을 하나의 토큰으로 만들어 나갑니다. 이것이 GPT 같은 LLM이 사용하는 토크나이저의 기본 원리입니다.

- GPT-2는 바이트 페어 인코딩(BPE) 토크나이저를 사용합니다.
- 어휘사전에 없는 단어를 더 작은 부분단어나 개별 문자로 분할하여 처리할 수 있습니다.
- 예를 들어 GPT-2의 어휘사전에 단어 "unfamiliarword"가 없다면 이를 ["unfam", "iliar", "word"] 같이 토큰화할 수 있습니다. BPE의 훈련에 따라 결과가 달라질 수 있습니다.
- 원본 BPE 토크나이저는 다음 주소에 있습니다: [https://github.com/openai/gpt-2/blob/master/src/encoder.py](https://github.com/openai/gpt-2/blob/master/src/encoder.py)
- 이 장에서는 오픈AI의 오픈 소스 [tiktoken](https://github.com/openai/tiktoken) 라이브러리에서 제공하는 BPE 토크나이저를 사용합니다. 이 라이브러리는 계산 성능을 높이기 위해 핵심 알고리즘을 러스트(Rust)로 구현했습니다.
- [../02_bonus_bytepair-encoder](../02_bonus_bytepair-encoder) 폴더에 두 토크나이저 구현을 비교하는 노트북이 있습니다(tiktoken이 약 5배 빠릅니다).

In [30]:
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")

In [31]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [32]:
tokenizer.special_tokens_set

{'<|endoftext|>'}

In [33]:
print(tokenizer.encode(text, allowed_special='all'))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [34]:
strings = tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


- BPE 토크나이저는 알지 못하는 단어를 부분단어나 개별 문자로 분할합니다:

<img src="images/llm_from_scratch/ch02_compressed/11.webp" width="600px">

## 2.6 슬라이딩 윈도로 데이터 샘플링하기

- 한 번에 한 단어씩 생성하도록 LLM을 훈련하므로 시퀀스에 있는 다음 단어가 예측할 타깃이 되도록 훈련 데이터를 준비합니다:

<img src="images/llm_from_scratch/ch02_compressed/12.webp" width="600px">

In [35]:
with open("datas/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


- 텍스트 청크에 대해 입력과 타깃이 있어야 합니다.
- 모델이 다음 단어를 예측해야 하므로 타깃은 오른쪽으로 한 토큰 이동한 입력입니다.

In [36]:
enc_sample = enc_text[50:]

In [37]:
context_size = 4

x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

print(f"x: {x}")
print(f"y:      {y}")

x: [290, 4920, 2241, 287]
y:      [4920, 2241, 287, 257]


- 한 번에 하나씩 예측은 다음과 같이 수행됩니다:

In [38]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [39]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


- 이후 장에서 어텐션 메커니즘을 소개한 후 다음 단어 예측을 다루겠습니다.
- 지금은 입력 데이터셋을 순회하면서 입력과 (한 토큰씩 뒤쳐진) 타깃을 반환하는 간단한 데이터 로더를 구현합니다.

- 슬라이딩 윈도를 사용해 한 토큰씩 위치를 이동합니다:

<img src="images/llm_from_scratch/ch02_compressed/13.webp" width="600px">

- 데이터셋과 입력 텍스트 데이터셋에서 청크를 추출하는 데이터 로더를 만듭니다.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        """
        Args:
            txt (str): 학습할 전체 텍스트 데이터
            tokenizer: 텍스트를 토큰 ID로 변환해주는 토크나이저 (예: tiktoken)
            max_length (int): 모델이 한 번에 볼 수 있는 윈도우 크기 (입력 시퀀스 길이)
            stride (int): 윈도우를 이동시킬 간격 (데이터 중복 정도를 결정)
        """
        self.input_ids = []
        self.target_ids = []

        # 1. 전체 텍스트 토큰화
        # 텍스트를 정수 리스트(token_ids)로 변환합니다.
        # <|endoftext|> 같은 특수 토큰도 허용하여 인코딩합니다.
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # 데이터가 너무 짧으면 학습할 수 없으므로 최소 길이를 확인합니다.
        assert len(token_ids) > max_length, "토큰화된 입력의 개수는 적어도 max_length+1과 같아야 합니다."

        # 2. 슬라이딩 윈도우(Sliding Window)로 데이터 생성
        # 전체 토큰 리스트를 훑으며 max_length 길이만큼 잘라냅니다.
        # stride만큼 건너뛰며 반복합니다.
        for i in range(0, len(token_ids) - max_length, stride):
            
            # 입력 청크: 현재 위치(i)부터 max_length만큼 가져옵니다.
            # 예: "나는 학교에"
            input_chunk = token_ids[i : i + max_length]
            
            # 타겟 청크: 입력보다 1칸 뒤의 위치(i+1)부터 가져옵니다.
            # GPT는 '다음 단어'를 맞추는 모델이므로, 정답은 입력보다 한 칸씩 뒤로 밀려있어야 합니다.
            # 예: "는 학교에 가서" ("나"-> "는", "는" -> "학교에" 예측)
            target_chunk = token_ids[i + 1 : i + max_length + 1]
            
            # 추출한 데이터를 텐서(Tensor)로 변환하여 리스트에 저장합니다.
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        # 데이터셋의 총 샘플(청크) 개수를 반환합니다.
        return len(self.input_ids)

    def __getitem__(self, idx):
        # DataLoader가 데이터를 요청할 때 호출됩니다.
        # 해당 인덱스(idx)의 입력과 정답 쌍을 반환합니다.
        return self.input_ids[idx], self.target_ids[idx]

In [ ]:
import tiktoken
from torch.utils.data import DataLoader

def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    """
    텍스트 데이터를 받아 학습용 데이터 로더(DataLoader)를 생성하는 함수입니다.

    Args:
        txt (str): 학습할 전체 텍스트 데이터 (raw string)
        batch_size (int): 한 번의 학습 단계(step)에 모델에 입력할 샘플의 개수
        max_length (int): 하나의 입력 시퀀스 길이 (Context Window 크기)
        stride (int): 윈도우를 이동시킬 간격 (이 값이 max_length보다 작으면 데이터가 겹침)
        shuffle (bool): 데이터를 섞을지 여부 (학습 시에는 True, 검증/평가 시에는 보통 False)
        drop_last (bool): 마지막 배치의 크기가 batch_size보다 작을 경우 버릴지 여부
        num_workers (int): 데이터 로딩에 사용할 CPU 프로세스 수 (0이면 메인 프로세스만 사용)

    Returns:
        dataloader (DataLoader): 배치 단위로 데이터를 반환하는 PyTorch DataLoader 객체
    """

    # 1. 토크나이저 초기화
    # 'gpt2' 모델이 사용하는 BPE(Byte Pair Encoding) 인코딩 방식을 불러옵니다.
    # 이 토크나이저는 텍스트를 정수 리스트(Token IDs)로 변환하는 역할을 합니다.
    tokenizer = tiktoken.get_encoding("gpt2")

    # 2. 데이터셋(Dataset) 생성
    # GPTDatasetV1 클래스(사용자가 정의한 클래스)를 사용하여 원본 텍스트를 처리합니다.
    # 이 단계에서 내부적으로 다음과 같은 작업이 일어납니다:
    #  - 전체 텍스트를 토큰화 (Integer Encoding)
    #  - max_length와 stride를 기반으로 슬라이딩 윈도우(Sliding Window) 적용
    #  - 입력 데이터(Input)와 타겟 데이터(Target) 쌍 생성 (Next Token Prediction용)
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # 3. 데이터 로더(DataLoader) 생성
    # PyTorch의 DataLoader는 위에서 만든 dataset을 사용하여 실제 학습에 쓸 배치를 만듭니다.
    dataloader = DataLoader(
        dataset,                # 데이터를 가져올 소스 (위에서 만든 데이터셋)
        batch_size=batch_size,  # 한 번에 가져올 데이터 묶음 크기 (예: 4개씩)
        shuffle=shuffle,        # 에폭(Epoch)마다 데이터 순서를 섞어 학습 편향 방지
        drop_last=drop_last,    # 마지막에 남는 짜투리 데이터(배치 크기보다 작은 경우)는 버림
        num_workers=num_workers # 병렬 처리를 위한 프로세스 개수 (Windows에서는 보통 0 권장)
    )

    return dataloader

- 문맥 크기를 4와 배치 크기 1로 데이터 로더를 테스트해 보죠.:

In [42]:
with open("datas/the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [43]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [44]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 367, 2885, 1464, 1807]]), tensor([[2885, 1464, 1807, 3619]])]


- 문맥 크기(이 경우 4)와 동일한 스트라이를 사용하는 예:

<img src="images/llm_from_scratch/ch02_compressed/14.webp" width="600px">

- 배치 출력도 만들 수 있습니다.
- 배치 간에 중첩이 있으면 과대적합이 증가될 수 있으므로 스트라이드를 증가시킵니다.

In [45]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("입력:\n", inputs)
print("\n타깃:\n", targets)

입력:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

타깃:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 토큰 임베딩 만들기

- LLM을 위한 데이터가 거의 준비되었습니다.
- 마지막으로 임베딩 층을 사용해 토큰을 연속적인 벡터 표현으로 임베딩해야 합니다.
- 일반적으로 이런 임베딩 층은 LLM의 일부이며 모델 훈련 과정에서 업데이트(훈련)됩니다.

<img src="images/llm_from_scratch/ch02_compressed/15.webp" width="500px">

- (토큰화를 거친 후) 토큰 ID가 2, 3, 5, 1인 샘플을 가정해 보죠:

In [46]:
input_ids = torch.tensor([2, 3, 5, 1])

- 간단한 예를 위해 어휘사전에는 6개 단어만 들어 있고, 임베딩 크기는 3이라고 가정해 보겠습니다:

In [47]:
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 결과적으로 6x3 가중치 행렬이 만들어집니다:

In [48]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


- 토큰 ID 3을 3차원 벡터로 변환하는 방법은 다음과 같습니다:

In [49]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


- 출력 결과는 `embedding_layer` 가중치 행렬에 있는 네 번째 행입니다.
- `input_ids`에 있는 네 개의 값을 다음과 같인 모두 임베딩할 수 있습니다.

In [50]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)


- 임베딩 층은 기본적으로 룩업 연산입니다:

<img src="images/llm_from_scratch/ch02_compressed/16.webp" width="700px">

- 임베딩 층과 일반적인 선형 층을 비교하고 싶다면 다음 노트북을 참고하세요: [../03_bonus_embedding-vs-matmul](../03_bonus_embedding-vs-matmul)

## 2.8 단어 위치 인코딩하기

- 임베딩 층은 토큰 ID를 입력 시퀀스에서 어떤 위치에 있던지 상관없이 동일한 벡터 표현으로 바꿉니다:

<img src="images/llm_from_scratch/ch02_compressed/17.webp" width="600px">

- 위치 임베딩을 토큰 임베딩에 더해서 대규모 언어 모델을 위한 입력 임베딩을 만듭니다:

<img src="images/llm_from_scratch/ch02_compressed/18.webp" width="600px">

- 바이트 페어 인코더의 어휘 사전 크기는 50,257입니다.
- 입력 토큰을 256차원의 벡터 표현으로 인코딩한다고 가정합니다.

In [ ]:
# 1. 환경 설정 및 하이퍼파라미터 정의
vocab_size = 50257  # 어휘 사전의 크기 (모델이 알 수 있는 총 단어/토큰의 개수)
output_dim = 256    # 임베딩 벡터의 차원 (한 단어를 256개의 숫자로 표현)

# 2. 토큰 임베딩 층 생성
# 단어의 인덱스(ID)를 받아서 고차원 벡터(256차원)로 변환하는 테이블입니다.
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

- 데이터 로더에서 데이터를 샘플링한 다음 배치에 있는 각 샘플의 토큰을 256차원 벡터로 임베딩할 수 있습니다.
- 배치 크기가 8이고 샘플마다 4개의 토큰이 있다면 8 x 4 x 256 크기의 텐서가 만들어 집니다.

In [ ]:
# 3. 데이터 로더 준비 및 샘플 데이터 추출
max_length = 4      # 한 번에 모델에 입력할 최대 문장 길이 (토큰 개수)
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)  # inputs 형태: [Batch_size, Max_length] -> [8, 4]

In [53]:
print("토큰 ID:\n", inputs)
print("\n입력 크기:\n", inputs.shape)

토큰 ID:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

입력 크기:
 torch.Size([8, 4])


In [ ]:
# 4. 토큰 임베딩 수행
# 각 단어 ID를 256차원의 벡터로 변환합니다.
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape) # 결과: torch.Size([8, 4, 256]) -> [배치 크기, 문장 길이, 임베딩 차원]

# 임베딩 벡터의 실제 숫자 값을 확인합니다.
print(token_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
         [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
         [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
         [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],

        [[ 1.5460,  1.7368, -0.7848,  ..., -0.1004,  0.8584, -0.3421],
         [-1.8622, -0.1914, -0.3812,  ...,  1.1220, -0.3496,  0.6091],
         [ 1.9847, -0.6483, -0.1415,  ..., -0.3841, -0.9355,  1.4478],
         [ 0.9647,  1.2974, -1.6207,  ...,  1.1463,  1.5797,  0.3969]],

        [[-0.7713,  0.6572,  0.1663,  ..., -0.8044,  0.0542,  0.7426],
         [ 0.8046,  0.5047,  1.2922,  ...,  1.4648,  0.4097,  0.3205],
         [ 0.0795, -1.7636,  0.5750,  ...,  2.1823,  1.8231, -0.3635],
         [ 0.4267, -0.0647,  0.5686,  ..., -0.5209,  1.3065,  0.8473]],

        ...,

        [[-1.6156,  0.9610, -2.6437,  ..., -0.9645,  1.0888,  1.6383],
         [-0.3985, -0.9235, -1.31

- GPT-2는 절대 위치 임베딩을 사용하므로 또 다른 임베딩 층을 만들면 됩니다:

In [ ]:
# 5. 위치 임베딩(Positional Embedding) 설정
# GPT 같은 트랜스포머 모델은 단어의 순서를 알지 못하므로, 위치 정보를 따로 학습시켜야 합니다.
context_length = max_length # 모델이 보는 문맥의 최대 길이
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

# 위치 임베딩 층의 초기 가중치(0~3번 위치에 해당하는 벡터들)를 확인합니다.
print(pos_embedding_layer.weight)

Parameter containing:
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       requires_grad=True)


In [ ]:
# 6. 위치 벡터 생성
# 0부터 max_length-1까지의 정수(0, 1, 2, 3)를 입력하여 각 위치에 해당하는 벡터를 가져옵니다.
# 위치 임베딩도 학습하게 됩니다.
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape) # 결과: torch.Size([4, 256])

# 생성된 위치 임베딩 값을 확인합니다.
print(pos_embeddings)

torch.Size([4, 256])
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       grad_fn=<EmbeddingBackward0>)


- LLM에 사용될 입력 임베딩을 만들기 위해 토큰 임베딩과 위치 임베딩을 더합니다:

In [ ]:
# 7. 최종 입력 임베딩 생성 (토큰 정보 + 위치 정보)
# [8, 4, 256] 텐서와 [4, 256] 텐서를 더합니다. (브로드캐스팅에 의해 모든 배치에 동일한 위치 정보가 더해짐)
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape) # 결과: torch.Size([8, 4, 256])

# 최종적으로 모델의 입력으로 들어갈 벡터 값을 확인합니다.
print(input_embeddings)

torch.Size([8, 4, 256])
tensor([[[ 2.2288,  0.5619,  0.8286,  ..., -0.6272, -0.2987,  0.8900],
         [ 2.0903, -0.4664, -0.0593,  ...,  0.9115, -1.0493, -1.6473],
         [-0.7158, -0.8304,  1.2494,  ...,  2.3952,  1.8773,  0.8051],
         [ 0.2703,  0.4029,  3.0514,  ...,  0.3595, -1.4548,  0.8310]],

        [[ 3.2835,  1.1749, -1.4150,  ..., -0.3281,  2.4332,  0.6924],
         [-0.2199, -0.9114, -0.1750,  ...,  1.5337, -0.1998,  0.1462],
         [ 1.5197, -1.4240,  0.4391,  ...,  1.0494, -1.4318,  2.3057],
         [ 0.2893,  0.8346, -0.1884,  ...,  1.9602,  0.8709,  0.8796]],

        [[ 0.9662,  0.0952, -0.4640,  ..., -1.0320,  1.6290,  1.7771],
         [ 2.4468, -0.2154,  1.4984,  ...,  1.8766,  0.5595, -0.1423],
         [-0.3856, -2.5393,  1.1556,  ...,  3.6157,  1.3267,  0.4944],
         [-0.2487, -0.5275,  2.0009,  ...,  0.2930,  0.5977,  1.3300]],

        ...,

        [[ 0.1219,  0.3991, -3.2740,  ..., -1.1921,  2.6637,  2.6728],
         [ 1.2438, -1.6436, -1.11

- 입력 전처리 워크플로의 초기 단계에서 입력 텍스트를 별개의 토큰으로 분할합니다.
- 이 분할 단계 다음에 사전에 정의된 어휘사전을 기반으로 토큰을 토큰 ID로 변환합니다.

<img src="images/llm_from_scratch/ch02_compressed/19.webp" width="600px">

## 2.9 Tokenizer 비교

- 아래 결과를 보면 한국어는 Exaone, 중국어는 Qwen의 Tokenizer 성능이 좋습니다

In [58]:
from transformers import AutoTokenizer
# Tokenizer 로드 (로컬 또는 온라인에서)
print("Tokenizer 로드 중...")
qwen = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")
exaone = AutoTokenizer.from_pretrained("LGAI-EXAONE/EXAONE-4.0-1.2B")

# 테스트할 텍스트
texts = [
    "안녕하세요, 반갑습니다!",
    "Hello, nice to meet you!",
    "인공지능 기술이 발전하고 있습니다.",
    "Hello! This is an English tokenizer test.",
    "こんにちは！日本語のテストです。",
    "人工智能技术正在快速发展。",
    "The quick brown fox jumps over the lazy dog. 빠른 갈색 여우가 게으른 개를 뛰어넘습니다.",
    "파이썬 프로그래밍은 매우 유용합니다. Python programming is very useful.",
    "123456789 !@#$%^&*() 특수문자",
]

print("\n" + "="*60)
for text in texts:
    print(f"\n텍스트: {text}")
    print("-"*60)
    
    # Qwen3
    qwen_tokens = qwen.encode(text, add_special_tokens=True)
    qwen_token_strs = [qwen.decode([tok]) for tok in qwen_tokens[:10]]
    print(f"Qwen3:  {len(qwen_tokens)}개 토큰")
    print(f"        {qwen_token_strs}")
    
    # EXAONE
    exaone_tokens = exaone.encode(text, add_special_tokens=True)
    exaone_token_strs = [exaone.decode([tok]) for tok in exaone_tokens[:10]]
    print(f"EXAONE: {len(exaone_tokens)}개 토큰")
    print(f"        {exaone_token_strs}")
    
    # 비교
    diff = len(qwen_tokens) - len(exaone_tokens)
    if diff > 0:
        print(f"→ EXAONE이 {diff}개 더 효율적")
    elif diff < 0:
        print(f"→ Qwen3가 {abs(diff)}개 더 효율적")
    else:
        print(f"→ 동일")

    # Special tokens 비교
print("\n" + "=" * 80)
print("4. Special Tokens 비교")
print("=" * 80)

print(f"\n[Qwen3 Special Tokens]")
for token_name in ['bos_token', 'eos_token', 'unk_token', 'pad_token']:
    token = getattr(qwen, token_name, None)
    if token:
        token_id = getattr(qwen, f"{token_name}_id", None)
        print(f"  {token_name}: '{token}' (ID: {token_id})")

print(f"\n[EXAONE Special Tokens]")
for token_name in ['bos_token', 'eos_token', 'unk_token', 'pad_token']:
    token = getattr(exaone, token_name, None)
    if token:
        token_id = getattr(exaone, f"{token_name}_id", None)
        print(f"  {token_name}: '{token}' (ID: {token_id})")

Tokenizer 로드 중...


텍스트: 안녕하세요, 반갑습니다!
------------------------------------------------------------
Qwen3:  8개 토큰
        ['안', '녕', '하세요', ',', ' 반', '갑', '습니다', '!']
EXAONE: 7개 토큰
        ['안녕', '하', '세요', ',', ' 반갑', '습니다', '!']
→ EXAONE이 1개 더 효율적

텍스트: Hello, nice to meet you!
------------------------------------------------------------
Qwen3:  7개 토큰
        ['Hello', ',', ' nice', ' to', ' meet', ' you', '!']
EXAONE: 7개 토큰
        ['Hello', ',', ' nice', ' to', ' meet', ' you', '!']
→ 동일

텍스트: 인공지능 기술이 발전하고 있습니다.
------------------------------------------------------------
Qwen3:  12개 토큰
        ['인', '공', '지', '능', ' 기', '술', '이', ' 발', '전', '하고']
EXAONE: 9개 토큰
        ['인공', '지능', ' 기술', '이', ' 발전', '하고', ' 있', '습니다', '.']
→ EXAONE이 3개 더 효율적

텍스트: Hello! This is an English tokenizer test.
------------------------------------------------------------
Qwen3:  9개 토큰
        ['Hello', '!', ' This', ' is', ' an', ' English', ' tokenizer', ' test', '.']
EXAONE: 9개 토큰
        ['Hello', 

## Hugging Face 모델 파일 구조 설명

Hugging Face Hub에 업로드된 파일들은 **Transformer 기반 언어 모델(LLM)**을 구성하는 핵심 요소들입니다. 크게 **설계도(Config)**, **가중치(Weights)**, **토크나이저(Tokenizer)** 세 그룹으로 나뉩니다.

---

### 1. 모델 아키텍처 및 설정 (설계도)

* **`config.json`**
    * **의미:** 모델의 **구조와 설계도**가 담긴 가장 중요한 설정 파일입니다.
    * **내용:** 모델의 종류(Llama, BERT 등), 레이어(Layer) 수, 히든 사이즈, 어텐션 헤드 수 등을 정의하여 빈 모델(Skeleton)을 만듭니다.

* **`generation_config.json`**
    * **의미:** 텍스트 **생성(Generation) 시 사용할 기본 옵션** 파일입니다.
    * **내용:** `max_length`(최대 길이), `temperature`(창의성), `top_p`, `eos_token_id`(종료 토큰) 등의 기본 설정값이 들어있습니다.

### 2. 모델 가중치 (학습된 뇌)

* **`model.safetensors`**
    * **의미:** 실제 **학습된 파라미터(가중치)**가 저장된 파일입니다. (가장 용량이 큼)
    * **특징:**
        * 기존 PyTorch의 `.bin` 파일보다 로딩 속도가 빠릅니다.
        * 보안 취약점(악성 코드 실행 등)을 해결한 안전한 형식입니다.

### 3. 토크나이저 (언어 처리기)
사람의 언어(Text)를 모델이 이해하는 숫자(Token ID)로 변환하는 규칙들입니다.

* **`vocab.json`**
    * **의미:** **단어장(Vocabulary)** 파일입니다. 단어와 그에 대응하는 숫자 ID가 매핑되어 있습니다.
* **`merges.txt`**
    * **의미:** **BPE 병합 규칙** 파일입니다. 알파벳 조각들이 어떻게 합쳐져서 단어가 되는지 정의합니다.
* **`tokenizer.json`**
    * **의미:** `vocab.json`과 `merges.txt` 등을 하나로 통합하여 최적화한 파일로, 로딩 속도가 매우 빠릅니다.
* **`tokenizer_config.json`**
    * **의미:** 토크나이저 설정 파일입니다. 어떤 클래스를 쓸지, 특수 토큰(`<s>`, `<pad>` 등)은 무엇인지 정의합니다.

---

### 요약

| 파일명 | 비유 | 역할 |
| :--- | :--- | :--- |
| `config.json` | **설계도** | 모델 구조 정의 |
| `model.safetensors` | **뇌** | 학습된 지식(가중치) 저장 |
| `tokenizer.json` | **번역기** | 텍스트 ↔ 숫자 변환 |
| `generation_config.json` | **말하기 습관** | 생성 옵션 설정 |

# 요약

이 장에서 구현한 데이터 로더의 간소화 버전이 [./dataloader.ipynb](./dataloader.ipynb) 노트북에 포함되어 있으며, 나중에 이어지는 장에서 GPT 모델을 훈련할 때 사용하겠습니다.

연습 문제 해답은 [./exercise-solutions.ipynb](./exercise-solutions.ipynb) 노트북을 참고하세요.

GPT-2의 토크나이저를 밑바닥부터 구현하고 훈련하는 방법이 궁금하다면 [바이트 페어 인코딩 토크나이저](../02_bonus_bytepair-encoder/compare-bpe-tiktoken.ipynb) 노트북을 참고하세요.